https://qiita.com/kamorits/items/6f342da395ad57468ae3

## マークダウンの色々
実行しないといけない
* test
* test
- test 

<font color="Red">テキスト</font>

In [1]:
import pandas as pd
import numpy as np

In [138]:
#訓練データの基礎情報
train=pd.read_csv("./train.csv",encoding = "utf8")
train.info()
train.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


訓練データのメモ

Name、Sex、Ticket、Cabin、Embarkedはobjectなので処理が必要
Sex、Embarkedはカテゴリ変数にする
NameとTicketから家族構成出せそうだけど手段が思いつかんので一旦放置。

Age、Cabin、Embarkedに欠損
Cabinは欠損多いから使えないかな…
Embarkedは件数少ないし該当レコードだけ消す
AgeはSexとPclassごとの中央値に後で置換する

In [139]:
#テストデータの基礎情報
test=pd.read_csv("./test.csv",encoding = "utf8")
test.info()
test.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


テストデータのメモ

Age、Cabin、Fareが欠損
Fareは欠損1件なんでdropna
残りは訓練データと同じ

仮説

◆生き残りやすい人とは
・上流階級、金持ち
・社会的弱者（女性、子供）

◆生き残りにくい人とは
・大人数（避難のしにくさ）

ステレオタイプとして、
ある程度裕福な家族は優先的に避難させてもらえそう。
貧乏男生の一人旅とかは生存率低そう、避難の手伝いとかさせられて最後まで残った挙句に沈むイメージ。
I'll be back....


◆方針
・欠損値の処理
Age：PclassとSexごとに中央値か平均値取って埋める
Embarked、Fare：該当レコードを削除
Cabin：使わない

・変数追加
何人の同乗者がいたのか（SibSp + Parch）
∟特に一人だったか否か（SibSp + Parch == 0 ）

・カテゴリ化
Sex、Embarked

・決定木、ランダムフォレスト、NN辺りでそれぞれ試してみる

・あと出来たら可視化もする

In [140]:
#Cabinの削除
del train["Cabin"]
del test["Cabin"]

In [141]:
#FareとEmbarkedの欠損処理
train = train.dropna(subset = ["Embarked"])
test = test.dropna(subset = ["Fare"])

In [103]:
"""使ってない
"""
def fill_missing_value(df,column,how=None,filled_value=None):
    """欠損値保管関数
    引数：
    　df：データ
    　column：補完対象のカラム
    　how：どうやって補完するか
    　filled_value：独自に計算した値を使用する場合
    返り値：
    　df：補完後データ
    """
    if filled_value==None:
        # 補完する数値を算出
        if how=="mean":
            filled_value = df[column].mean()
        elif how=="median":
            filled_value = df[column].median()
        elif how=="mode":
            filled_value = df[column].mode()
        else:
            filled_value = 0
    else:
        pass
    # 補完
    df.loc[df[column].isnull(),column]=filled_value
    return df

train = fill_missing_value(train,"Embarked","mode")
train = fill_missing_value(train,"Age","mean")
test = fill_missing_value(test,"Age","mean")
test = fill_missing_value(test,"Fare","median")


In [20]:
#階級と性別による年齢の中央値と平均値
train_m = train.groupby(["Pclass","Sex"]).agg(["median","mean"])
train_m

PassengerId             Survived              Age             \
                   median        mean   median      mean median       mean   
Pclass Sex                                                                   
1      female       447.0  469.717391      1.0  0.967391   35.0  34.240964   
       male         480.5  455.729508      0.0  0.368852   40.0  41.281386   
2      female       439.5  443.105263      1.0  0.921053   28.0  28.722973   
       male         416.5  447.962963      0.0  0.157407   30.0  30.740707   
3      female       376.0  399.729167      0.5  0.500000   21.5  21.750000   
       male         466.0  455.515850      0.0  0.135447   25.0  26.507589   

               SibSp            Parch               Fare              
              median      mean median      mean   median        mean  
Pclass Sex                                                            
1      female      0  0.565217      0  0.467391  83.1583  106.693750  
       male        0  0.311475      0  0.278689  41.2625   67.226127  
2      female      0  0.486842      0  0.605263  22.0000   21.970121  
       male        0  0.342593      0  0.222222  13.0000   19.741782  
3      female      0  0.895833      0  0.798611  12.4750   16.118810  
       male        0  0.498559      0  0.224784   7.9250   12.661633

In [104]:
"""使ってない
"""
#普通に平均値を入れるだけなら↓でいいっぽい（前任者のカンニングをした）
train["Age"][train["Age"].isnull()]=train_m["Age"].median()
test["Age"][test["Age"].isnull()]=train_m["Age"].median()


<ipython-input-104-e99079d9d0e7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["Age"][train["Age"].isnull()]=train_m["Age"].median()
<ipython-input-104-e99079d9d0e7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["Age"][test["Age"].isnull()]=train_m["Age"].median()


In [142]:
#前処理ぐらいならマジックナンバー使っていい
filled_value=[
    {"conditions":["male",1],"filled_value":36.0},
    {"conditions":["male",2],"filled_value":29.7},
    {"conditions":["male",3],"filled_value":29.7},
    {"conditions":["female",1],"filled_value":32.5},
    {"conditions":["female",2],"filled_value":28.5},
    {"conditions":["female",3],"filled_value":28.0},
]

for i in filled_value:
    print(i["conditions"][0],i["conditions"][1],i["filled_value"])
    train.loc[
        (
            (pd.isnull(train["Age"]))&
            (train["Sex"]==i["conditions"][0])&
            (train["Pclass"]==i["conditions"][1])
        )
    ,"Age"]=i["filled_value"]
    test.loc[
        (
            (pd.isnull(test["Age"]))&
            (test["Sex"]==i["conditions"][0])&
            (test["Pclass"]==i["conditions"][1])
        )
    ,"Age"]=i["filled_value"]

male 1 36.0
male 2 29.7
male 3 29.7
female 1 32.5
female 2 28.5
female 3 28.0


・変数追加
何人の同乗者がいたのか（SibSp + Parch）
∟特に一人だったか否か（SibSp + Parch == 0 ）

これがまだ出来ていない

In [143]:
train["passenger"] = train["SibSp"] + train["Parch"]
train["is_one"] = train["SibSp"]+train["Parch"]==0
test["passenger"] = test["SibSp"] + test["Parch"]
test["is_one"] = test["SibSp"]+test["Parch"]==0

In [144]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,passenger,is_one
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,1,False
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,1,False
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,0,True
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,1,False
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,S,0,True
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,S,0,True
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,28.0,1,2,W./C. 6607,23.4500,S,3,False
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C,0,True


In [124]:
train["is_one"] = train["is_one"].astype(int)

In [125]:
#変数化
train = pd.get_dummies(train[["Age","Survived","Sex","Pclass","SibSp","Parch","Fare","Embarked","passenger","is_one"]])
test = pd.get_dummies(test[["Age","Sex","Pclass","SibSp","Parch","Fare","Embarked","passenger","is_one"]])
train

,Age,Survived,Pclass,SibSp,Parch,Fare,passenger,is_one,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,22.0,0,3,1,0,7.2500,1,0,0,1,0,0,1
1,38.0,1,1,1,0,71.2833,1,0,1,0,1,0,0
2,26.0,1,3,0,0,7.9250,0,1,1,0,0,0,1
3,35.0,1,1,1,0,53.1000,1,0,1,0,0,0,1
4,35.0,0,3,0,0,8.0500,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,27.0,0,2,0,0,13.0000,0,1,0,1,0,0,1
887,19.0,1,1,0,0,30.0000,0,1,1,0,0,0,1
888,28.0,0,3,1,2,23.4500,3,0,1,0,0,0,1
889,26.0,1,1,0,0,30.0000,0,1,0,1,1,0,0


In [126]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 417 entries, 0 to 417
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Age         417 non-null    float64
 1   Pclass      417 non-null    int64  
 2   SibSp       417 non-null    int64  
 3   Parch       417 non-null    int64  
 4   Fare        417 non-null    float64
 5   passenger   417 non-null    int64  
 6   is_one      417 non-null    bool   
 7   Sex_female  417 non-null    uint8  
 8   Sex_male    417 non-null    uint8  
 9   Embarked_C  417 non-null    uint8  
 10  Embarked_Q  417 non-null    uint8  
 11  Embarked_S  417 non-null    uint8  
dtypes: bool(1), float64(2), int64(4), uint8(5)
memory usage: 25.2 KB


In [127]:
from sklearn.model_selection import train_test_split

X_train,X_validation_test,y_train,y_validation_test = train_test_split(train[["Sex_female","Sex_male","Pclass","SibSp","Parch","Fare","Embarked_C","Embarked_S","Embarked_Q","passenger","is_one"]],train["Survived"],test_size=0.3,random_state=1)

In [128]:
from sklearn.neural_network import MLPClassifier 
clf = MLPClassifier(random_state=1)

In [129]:
from scipy.stats import zscore
X_train = zscore(Xtrain,axis=1)
X_validation_test = zscore(Xtest,axis=1)

In [130]:
clf.fit(X_train,y_train)

c:\users\mitsuyoshi.toki\appdata\local\programs\python\python38\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(random_state=1)

In [131]:
#↑のエラーの理由がわからん…NaNはない、floatはFareだけだけど大きいわけじゃない、無限大もない。
train.describe()

,Age,Survived,Pclass,SibSp,Parch,Fare,passenger,is_one,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
count,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000
mean,29.747773,0.382452,2.311586,0.524184,0.382452,32.096681,0.906637,0.601800,0.350956,0.649044,0.188976,0.086614,0.724409
std,13.012933,0.486260,0.834700,1.103705,0.806761,49.697504,1.614703,0.489803,0.477538,0.477538,0.391710,0.281427,0.447063
min,0.420000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,22.000000,0.000000,2.000000,0.000000,0.000000,7.895800,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,29.700000,0.000000,3.000000,0.000000,0.000000,14.454200,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000
75%,36.000000,1.000000,3.000000,1.000000,0.000000,31.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000
max,80.000000,1.000000,3.000000,8.000000,6.000000,512.329200,10.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [132]:
predicted = clf.predict(X_validation_test)

In [133]:
print(predicted)

[1 1 0 1 0 1 1 1 0 1 1 1 0 1 0 1 0 1 1 1 1 1 0 0 1 1 1 0 0 1 1 1 1 1 0 1 1
 0 1 1 1 0 1 0 0 1 1 0 1 0 0 1 0 1 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 1 1 1 0 0
 1 1 0 0 0 0 0 0 1 1 0 0 1 1 0 1 0 1 0 0 1 0 1 1 1 1 0 1 0 0 0 0 1 0 0 0 1
 0 0 1 1 1 0 1 1 0 0 1 1 0 1 0 1 0 0 1 1 0 0 0 1 1 0 0 0 0 0 1 0 1 0 0 1 1
 1 0 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 1 1 1 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1
 1 1 0 1 0 1 1 0 0 0 0 0 1 0 1 1 1 0 0 1 0 0 1 0 1 0 0 0 0 1 1 0 1 0 1 0 1
 0 1 1 0 0 0 0 0 1 1 1 0 0 0 0 0 1 0 1 0 0 0 1 1 1 1 0 0 1 1 1 0 1 1 0 0 1
 0 1 0 1 0 1 0 0]


In [134]:
print(sum(predicted==y_test)/len(y_test))

0.7865168539325843


https://github.com/git-for-windows/git/releases/download/v2.27.0.windows.1/Git-2.27.0-64-bit.exe
gitのインストールをした

